# Gromos Tutorial Pipeline

In [3]:
import os, sys
root_dir = os.getcwd()

#if package is not installed and path not set correct - this helps you out :)
sys.path.append(root_dir+"/..")

C:\Users\benja\OneDrive - ETH Zürich\PhD\Code\pygromos\examples

GROMOSPATH 


In [ ]:
print(orig_dir)
from pygromos.gromos.gromosPP import GromosPP
from pygromos.gromos.gromosXX import GromosXX

gromosPP_bin = None
gromosXX_bin = None
gromPP = GromosPP(gromosPP_bin)
gromXX = GromosXX(gromosXX_bin)
gromPP = GromosPP()

## generate Topology
### build single topologies

In [ ]:
from pygromos.data.ff import Gromos54A7

## Make Cl-
sequence = "CL-"
solvent = "H2O"
top_cl = "example_files/topo/cl.top"

gromPP.make_top(in_building_block_lib_path=Gromos54A7.mtb, in_parameter_lib_path=Gromos54A7.ifp,
                in_sequence=sequence, in_solvent="",out_top_path=top_cl)

## Make Peptide
sequence = "NH3+ VAL TYR ARG LYSH GLN COO-"
solvent = "H2O"
top_peptide = "example_files/topo/peptide.top"

gromPP.make_top(in_building_block_lib_path=Gromos54A7.mtb, in_parameter_lib_path=Gromos54A7.ifp,
                in_sequence=sequence, in_solvent="",out_top_path=top_peptide)


### combine topology

In [ ]:
top_vacuum_system = "example_files/topo/vac_sys.top"
gromPP.com_top(top_peptide, top_cl, out_top_path=top_vacuum_system)


## generate coordinates


In [ ]:
in_pdb = "example_files/coord/peptide.pdb"
cnf_peptide = "examples/example_files/coord/cnf_vacuum_peptide.cnf"

cnf_peptide = gromPP.pdb2gromos(in_pdb_path=in_pdb, in_top_path=top_peptide, out_cnf_path=cnf_peptide)

### add hydrogens

In [ ]:
cnf_hpeptide = "example_files/coord/cnf_vacuum_hpeptide.cnf"
cnf_hpeptide = gromPP.protonate(in_cnf_path=cnf_peptide, in_top_path=top_peptide, out_cnf_path=cnf_hpeptide)

### cnf to pdb

In [ ]:
out_pdb = "example_files/coord/cnf_vacuum_hpeptide.pdb"
out_pdb = gromPP.frameout(in_coord_path=cnf_hpeptide, in_top_path=top_peptide, out_file_path=out_pdb,
                               periodic_boundary_condition="v", out_file_format="pdb", notimeblock=True)

### Vacuum energy minimization

In [ ]:
from pygromos.data.imd_templates import template_emin
from pygromos.files import imd

vacuum_emin_dir = "example_files/box"
out_prefix = "vacuum_emin"

vacuum_emin_imd_file = imd.Imd(template_emin)
imd_emin_vacuum = vacuum_emin_imd_file.write(vacuum_emin_dir+"/"+out_prefix+".imd")
os.chdir(vacuum_emin_dir)


cnf_emin_vacuum = vacuum_emin_dir + "/" + out_prefix+".cnf"

### build box system

In [ ]:
from pygromos.data.solvents import spc
cnf_box = gromPP.sim_box(in_top_path=top_vacuum_system, in_cnf_path=cnf_emin_vacuum,in_solvent_cnf_file_path=spc,
                         periodic_boundary_condition="r", minwall=0.8, threshold=0.23, rotate=True)

## run Energy Minimization

In [ ]:
from pygromos.files import imd
from pygromos.files.coord import cnf
from pygromos.data.imd_templates import template_emin

emin_dir = "example_files/min"
out_prefix = "emin"

cnf_box = cnf.Cnf(input=cnf_box)
residues = cnf_box.get_residues()

emin_imd = imd.Imd(template_emin)
imd_emin_box = emin_imd.write(emin_dir+"/"+out_prefix+".imd")
#TODO: add solv num
cnf_reference_position = cnf_box.write_refpos(out_prefix+"/refpos.rpf")
cnf_position_restraint = cnf_box.write_possrespec(out_prefix+"/posres.pos")

os.chdir(emin_dir)

gromXX.md_run(in_coord_path=cnf_box, in_topo_path=top_vacuum_system,
              in_refpos_path=cnf_reference_position, cnf_position_restraint=cnf_position_restraint,
              in_imd_path=imd_emin_box,
              out_prefix=out_prefix)


## Equilibration
 To be implemented!

In [6]:
from pygromos.files import imd
from pygromos.files.coord import cnf
from pygromos.data.imd_templates import template_emin

emin_dir = "example_files/min"
out_prefix = "emin"

emin_imd = imd.Imd(template_emin)
emin_imd.STEP.NSTLIM = 10000
imd_emin_box = emin_imd.write(emin_dir+"/"+out_prefix+".imd")

cnf_box = cnf.Cnf(input=cnf_box)
residues = cnf_box.get_residues()
out = emin_imd.clean_residue_list_for_imd(residues)


New file generated: example_files/min/emin.imd


NameError: name 'cnf_box' is not defined